In [25]:
import time
import socket

from random import randint

from io import BytesIO
from helper import (
    little_endian_to_int,
    int_to_little_endian,
    read_varint,
    hash256,
    encode_varint,)
from block import Block

MAINNET_NETWORK_MAGIC = b'\xf9\xbe\xb4\xd9'
TESTNET_NETWORK_MAGIC = b'\x0b\x11\x09\x07'

class NetworkEnvelope:
    
    def __init__(self, command, payload, testnet=False):
        self.command = command # human-readable command, 12 bytes
        self.payload = payload
        if testnet:
            self.magic = TESTNET_NETWORK_MAGIC
        else:
            self.magic = MAINNET_NETWORK_MAGIC
        
    def __repr__(self):
        return f'{self.command}: {self.payload.hex()}'
    
    @classmethod
    def parse(cls, s, testnet=False):
        magic = s.read(4)
        if magic == b'':
            raise RuntimeError('Connection reset!')
        if testnet:
            expected_magic = TESTNET_NETWORK_MAGIC
        else:
            expected_magic = MAINNET_NETWORK_MAGIC
        if magic != expected_magic:
            raise RuntimeError('Magic is wrong')
        command = s.read(12)
        command = command.strip(b'\x00')
        payload_len = little_endian_to_int(s.read(4))
        payload_checksum = s.read(4)
        payload = s.read(payload_len)
        if hash256(payload)[:4] != payload_checksum:
            raise RuntimeError('Checksum does not match')
        return cls(command, payload, testnet)
    
    def serialize(self):
        result = self.magic
        result += self.command + b'\x00' * (12 - len(self.command))
        result += int_to_little_endian(len(self.payload), 4)
        result += hash256(self.payload)[:4]
        result += self.payload
        return result
    
    def stream(self):
        return BytesIO(self.payload)
    
class VersionMessage:
    
    command = b'version'
    
    def __init__(self, version=70015, services=0, timestamp=None,
                receiver_services=0,
                receiver_ip=b'\x00\x00\x00\x00', receiver_port=8333,
                sender_services=0,
                sender_ip=b'\x00\x00\x00\x00', sender_port=8333,
                nonce=None, user_agent=b'/programmingbitcoin:0.1/',
                latest_block=0, relay=False):
        self.version = version
        self.services = services
        if timestamp is None:
            self.timestamp = int(time.time())
        else:
            self.timestamp = timestamp
        self.receiver_services = receiver_services
        self.receiver_ip = receiver_ip
        self.receiver_port = receiver_port
        self.sender_services = sender_services
        self.sender_ip = sender_ip
        self.sender_port = sender_port
        if nonce is None:
            self.nonce = int_to_little_endian(randint(0, 2**64), 8)
        else:
            self.nonce = nonce
        self.user_agent = user_agent
        self.latest_block = latest_block
        self.relay = relay
        
    def serialize(self):
        result = int_to_little_endian(self.version, 4)
        result += int_to_little_endian(self.services, 8)
        result += int_to_little_endian(self.timestamp, 8)
        result += int_to_little_endian(self.receiver_services, 8)
        result += b'\x00' * 10 + b'\xff\xff' + self.receiver_ip
        result += self.receiver_port.to_bytes(2, 'big')
        result += int_to_little_endian(self.sender_services, 8)
        result += b'\x00' * 10 + b'\xff\xff' + self.sender_ip
        result += self.sender_port.to_bytes(2, 'big')
        result += self.nonce
        result += encode_varint(len(self.user_agent))
        result += self.user_agent
        result += int_to_little_endian(self.latest_block, 4)
        if self.relay:
            result += b'\x01'
        else:
            result += b'\x00'
        return result
    
class VerAckMessage:
    command = b'verack'
    
    def __init__(self):
        pass
    
    @classmethod
    def parse(cls, s):
        return cls()
    
    def serialize(self):
        return b''
    
class PingMessage:
    command = b'ping'

    def __init__(self, nonce):
        self.nonce = nonce

    @classmethod
    def parse(cls, s):
        nonce = s.read(8)
        return cls(nonce)

    def serialize(self):
        return self.nonce


class PongMessage:
    command = b'pong'

    def __init__(self, nonce):
        self.nonce = nonce

    def parse(cls, s):
        nonce = s.read(8)
        return cls(nonce)

    def serialize(self):
        return self.nonce
    
class SimpleNode:
    
    def __init__(self, host, port=None, testnet=False, logging=False):
        if port is None:
            if testnet:
                port = 18333
            else:
                port = 8333
        self.testnet = testnet
        self.logging = logging
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((host, port))
        self.stream = self.socket.makefile('rb', None)
        
    def send(self, message):
        envelope = NetworkEnvelope(
            message.command, message.serialize(), testnet=self.testnet)
        if self.logging:
            print(f'sending: {envelope}')
        self.socket.sendall(envelope.serialize())
        
    def read(self):
        envelope = NetworkEnvelope.parse(self.stream, testnet=self.testnet)
        if self.logging:
            print(f'receiving: {envelope}')
        return envelope
    
    def wait_for(self, *message_classes):
        command = None
        command_to_class = {m.command: m for m in message_classes}
        while command not in command_to_class.keys():
            envelope = self.read()
            command = envelope.command
            if command == VersionMessage.command:
                self.send(VerAckMessage())
            elif command == PingMessage.command:
                self.send(PongMessage(envelope.payload))
        return command_to_class[command].parse(envelope.stream())
    
    def handshake(self):
        version = VersionMessage()
        self.send(version)
        self.wait_for(VerAckMessage)
        
class GetHeadersMessage:
    command = b'getheaders'
    
    def __init__(self, version=70015, num_hashes=1,
                start_block=None, end_block=None):
        self.version = version
        self.num_hashes = num_hashes
        if start_block is None:
            raise RuntimeError('a start block is required')
        self.start_block = start_block
        if end_block is None:
            self.end_block = b'\x00' * 32
        else:
            self.end_block = end_block
            
    def serialize(self):
        result = int_to_little_endian(self.version, 4)
        result += encode_varint(self.num_hashes)
        result += self.start_block[::-1]
        result += self.end_block[::-1]
        return result
    
class HeadersMessage:
    command = b'headers'
    
    def __init__(self, blocks):
        self.blocks = blocks
        
    @classmethod
    def parse(cls, s):
        num_headers = read_varint(s)
        blocks = []
        for _ in range(num_headers):
            blocks.append(Block.parse(s))
            num_txs = read_varint(s)
            if num_txs != 0:
                raise RuntimeError('num of txs not 0')
        return cls(blocks)
        

In [26]:
message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'
msg = NetworkEnvelope.parse(BytesIO(bytes.fromhex(message_hex)))
print(msg)
print(b'Hello'.hex())
a = {'1':2}
a.keys()

b'verack': 
48656c6c6f


dict_keys(['1'])

In [31]:
from io import BytesIO
# from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
for _ in range(30):
    getheaders = GetHeadersMessage(start_block=previous.hash256())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow():
            raise RuntimeError('bad PoW at block {}'.format(count))
        if header.prev_block_hash != previous.hash256():
            raise RuntimeError('discontinuous block at {}'.format(count))
        if count % 2016 == 0:
            time_diff = previous.timestamp - first_epoch_timestamp
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex(),", ",header.hash256()[-4:])
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        previous = header
        count += 1

ffff001d ,  b'l{\xdd\xef'
ffff001d ,  b'4] \xd0'
ffff001d ,  b'\xf1\xdc\xb2\x13'
ffff001d ,  b'\xb6F\xb9,'
ffff001d ,  b'V\x15@S'
ffff001d ,  b'\xa5\xffc\xe0'
ffff001d ,  b'?\xe0\xd2\xc4'
ffff001d ,  b'(\xff%R'
ffff001d ,  b'\xe9\x9b\x01\xfe'
ffff001d ,  b'VmF\x1e'
ffff001d ,  b'\xa1p\x1d\xe0'
ffff001d ,  b'\x9c\xf4m\xf2'
ffff001d ,  b'\xc6\xd0\x07\xcc'
ffff001d ,  b'Z\xcd\xd9|'
ffff001d ,  b'\\\x7f\xb0\xa8'
6ad8001d ,  b'\xe0\x90Ig'
28c4001d ,  b'\x0bY\x19\xea'
71be001d ,  b'\xb0YK2'
c38c001d ,  b'\xc8S2\x8a'
5746651c ,  b'$\rrE'
e5b3431c ,  b'/\xba\x96\x12'
6f7f381c ,  b'\xc0\xf0\xd7\xa3'
7513381c ,  b'm]\x96\xc6'
15112a1c ,  b'\xf7\xd7\xd8\xfc'
a7bc201c ,  b's\xbe\x07\xc9'
6f54161c ,  b'\xe9\xa3\xabc'
53ec131c ,  b'\xb5\x88w\xd6'
249c151c ,  b'\x8er\xf4\xed'
5c670f1c ,  b'$\xf9\xb7\x99'
